<a href="https://colab.research.google.com/github/rbawden/Tutoriel-Normalisation/blob/main/Tutoriel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installer les paquets nécessaires

In [30]:
!pip install gdown==4.2.0 fairseq sentencepiece sacrebleu hydra-core
!pip install omegaconf==2.1.1

  Using cached omegaconf-1.4.1-py3-none-any.whl (14 kB)
  Attempting uninstall: omegaconf
    Found existing installation: omegaconf 2.1.1
    Uninstalling omegaconf-2.1.1:
      Successfully uninstalled omegaconf-2.1.1
  Using cached omegaconf-2.1.1-py3-none-any.whl (74 kB)
  Attempting uninstall: omegaconf
    Found existing installation: omegaconf 1.4.1
    Uninstalling omegaconf-1.4.1:
      Successfully uninstalled omegaconf-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hydra-core 0.11.3 requires omegaconf<1.5,>=1.4, but you have omegaconf 2.1.1 which is incompatible.


In [66]:
!python --version

Python 3.8.5


Télécharger les données et les modèles depuis Google Drive

In [32]:
![ -d data-models ] || gdown https://drive.google.com/drive/folders/1h-qSnPBPZFZQ_kqWIBMhkkFS-6C2b10H?usp=sharing -O data-models --folder 

Créer des liens symboliques pour rendre les fichiers/dossiers téléchargés plus accessible

In [54]:
![ -d data ] || mkdir data; [ -d models ] || mkdir models
!for file in data-models/*{src,trg,model,vocab}; do ln -sf  $PWD/$file $PWD/data/; done
!for file in data-models/*{txt,npz}; do ln -sf $PWD/$file $PWD/models/; done

In [55]:
import os, sys
for folder in 'models', 'data':
  path = os.path.abspath(os.path.join('./', folder))
  if path not in sys.path:
    sys.path.append(path)

Fonction pour lire le contenu d'un fichier ligne par ligne

In [56]:
def lire_fichier(nom_fichier):
  contenu = []
  with open(nom_fichier) as fp:
    for line in fp:
      contenu.append(line.strip())
  return contenu

def ecrire_fichier(liste_phrases, nom_fichier):
    with open(nom_fichier, 'w') as fp:
        for phrase in liste_phrases:
            fp.write(phrase + '\n')

Lire le contenu des données parallèles

In [57]:
data_src = lire_fichier(os.getcwd() + '/data/dev.src')
data_trg = lire_fichier('data/dev.trg')

Visualiser les fichiers sources et cibles

In [58]:
for i in range(4):
    print('src = ', data_src[i])
    print('trg = ', data_trg[i])
    print('--')

src =  1.
trg =  1.
--
src =  1. QVe cette propoſtion, qu'vn eſpace eſt vuidé, repugne au ſens commun.
trg =  1. QUe cette proposition, qu'un espace est vidé, répugne au sens commun.
--
src =  1. QVe tous les corps ont repugnance à ſe ſeparer l'vn de l'autre, & admettre du vuide dans leur interualle;
trg =  1. QUe tous les corps ont répugnance à se séparer l'un de l'autre, et admettre du vide dans leur intervalle;
--
src =  1. QVe tous les corps ont repugnance à ſe ſeparer l'vn de l'autre, & admettre ce vuide apparent dans leur interualle:
trg =  1. QUe tous les corps ont répugnance à se séparer l'un de l'autre, et admettre ce vide apparent dans leur intervalle:
--


Charger le modèle de segmentation en sous-mots

In [59]:
import sentencepiece
spm = sentencepiece.SentencePieceProcessor(model_file='data/bpe_joint_1000.model')

Appliquer le modèle sur les données

In [60]:
data_src_sp = spm.encode(data_src, out_type=str)
data_trg_sp = spm.encode(data_trg, out_type=str)

Ecrire les données pre-traités dans des fichiers

Visualiser les fichiers

In [61]:
ecrire_fichier([' '.join(phrase) for phrase in data_src_sp], 'data/dev.sp.src')
ecrire_fichier([' '.join(phrase) for phrase in data_trg_sp], 'data/dev.sp.trg')

In [62]:
for i in range(4):
    print('src = ', data_src_sp[i])
    print('trg = ', data_trg_sp[i])
    print('--')

src =  ['▁1', '.']
trg =  ['▁1', '.']
--
src =  ['▁1', '.', '▁Q', 'V', 'e', '▁cette', '▁prop', 'ost', 'ion', ',', '▁qu', "'", 'vn', '▁esp', 'ace', '▁est', '▁v', 'ui', 'd', 'é', ',', '▁re', 'p', 'u', 'gne', '▁au', '▁sens', '▁comm', 'un', '.']
trg =  ['▁1', '.', '▁Q', 'U', 'e', '▁cette', '▁prop', 'os', 'it', 'ion', ',', '▁qu', "'", 'un', '▁esp', 'ace', '▁est', '▁v', 'id', 'é', ',', '▁rép', 'u', 'gne', '▁au', '▁sens', '▁comm', 'un', '.']
--
src =  ['▁1', '.', '▁Q', 'V', 'e', '▁tous', '▁les', '▁cor', 'p', 's', '▁ont', '▁re', 'p', 'u', 'gn', 'ance', '▁à', '▁se', '▁se', 'p', 'are', 'r', '▁l', "'", 'vn', '▁de', '▁l', "'", 'autre', ',', '▁&', '▁ad', 'm', 'ettre', '▁du', '▁v', 'ui', 'de', '▁dans', '▁leur', '▁in', 'ter', 'u', 'al', 'le', ';']
trg =  ['▁1', '.', '▁Q', 'U', 'e', '▁tous', '▁les', '▁cor', 'p', 's', '▁ont', '▁rép', 'u', 'gn', 'ance', '▁à', '▁se', '▁s', 'ép', 'are', 'r', '▁l', "'", 'un', '▁de', '▁l', "'", 'autre', ',', '▁et', '▁ad', 'm', 'ettre', '▁du', '▁vi', 'de', '▁dans', '▁leur', 

Définir une fonction pour détokeniser (pour plus tard)

In [63]:
def decode_sp(list_text):
    return [''.join(sent).replace('▁', ' ') for sent in list_text]

In [64]:
decode_sp(data_src_sp[:5])

[' 1.',
 " 1. QVe cette propostion, qu'vn espace est vuidé, repugne au sens commun.",
 " 1. QVe tous les corps ont repugnance à se separer l'vn de l'autre, & admettre du vuide dans leur interualle;",
 " 1. QVe tous les corps ont repugnance à se separer l'vn de l'autre, & admettre ce vuide apparent dans leur interualle:",
 ' 2.']

Charger le modèle de normalisation déjà entraîné

In [65]:
!cat data/dev.sp.src | fairseq-interactive models/best_lstm --path models/best_lstm/model.npz > data/dev.sp.norm.src

Traceback (most recent call last):
  File "/Users/rbawden/miniconda3/bin/fairseq-interactive", line 5, in <module>
    from fairseq_cli.interactive import cli_main
  File "/Users/rbawden/miniconda3/lib/python3.8/site-packages/fairseq_cli/interactive.py", line 20, in <module>
    from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils
  File "/Users/rbawden/miniconda3/lib/python3.8/site-packages/fairseq/__init__.py", line 20, in <module>
    import fairseq.models  # noqa
  File "/Users/rbawden/miniconda3/lib/python3.8/site-packages/fairseq/models/__init__.py", line 177, in <module>
    module = importlib.import_module("fairseq.models." + model_name)
  File "/Users/rbawden/miniconda3/lib/python3.8/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "/Users/rbawden/miniconda3/lib/python3.8/site-packages/fairseq/models/transformer_lm.py", line 10, in <module>
    from fairseq import options, utils
